In [3]:
from transformers import pipeline, set_seed
set_seed(42)

In [4]:
generator = pipeline('text-generation', model='gpt2')
generator("", max_length=100, num_return_sequences=10)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "As if that wasn't enough, the team is also making a few big changes to the game with the launch of the new Star Wars mobile app. Star Wars Battlefront: Rebellion on the Xbox Play Store will also feature new games based on existing content from the series, including Galactic Core and the next instalment in Rebels: Knights of the Old Republic. In addition, new games will be available on the Xbox One, PC, and PS4, a major increase in the amount of content"},
 {'generated_text': '- - - - - - - - -\n\nROBIN ROSS - - - - - - - - -\n\nI am now going to give this one a shot.\n\nWhat is this thing and why are two of my friends so so different!\n\nROBIN - - - - - - - - - - - -\n\nWhy does my hair look so different?\n\nJUAN GONZÁLEZ - - - - -'},
 {'generated_text': '\nThe new study says that there is no difference between male and female sex chromosomes.\n\nGender equality. - Professor Sainath Gupta\n\nThe National Research Council has made great progress on that front, descr

In [6]:
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')
generator("", max_length=100, num_return_sequences=10)

Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/200 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "// @flow\n\nimport type {\n  types as typesArray\n} from 'common/typescript/types';\n\ntype typeOf = typesArray['default'](3, 4);\n\ntype classArray = typesArray['class'](2, 3);\n\ntype typeOfType = {\n  typeOf: typeOf\n  [type: string]: type\n};\n\ntype getters = {\n  get: typeOfType[Object"},
 {'generated_text': 'Q:\n\nHow can I create a list of tuples or tuples of tuples in python?\n\nI can\'t find how to create a list of tuples in python, can someone help me?\n>>> def create_list(list_of_tuples):\n        """ Create a list [element1, element2, element3, etc. ] """\n        # print the list.\n '},
 {'generated_text': 'The invention relates to a method for manufacturing a semiconductor integrated device and, more particularly, to a method for manufacturing a semiconductor integrated device including a source/drain diffusion layer and a semiconductor layer located between the source/drain diffusion layer and the semiconductor layer being exposed into active region

In [8]:
generator = pipeline('text-generation', model='distilbert') # Not correct yet
generator("", max_length=100, num_return_sequences=10)

404 Client Error: Not Found for url: https://huggingface.co/distilbert/resolve/main/config.json


OSError: Can't load config for 'distilbert'. Make sure that:

- 'distilbert' is a correct model identifier listed on 'https://huggingface.co/models'
  (make sure 'distilbert' is not a path to a local directory with something else, in that case)

- or 'distilbert' is the correct path to a directory containing a config.json file

